In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import random
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

#!pip install psycopg2-binary
%reload_ext sql

In [3]:
bikeshare_data = pd.read_csv("202203-capitalbikeshare-tripdata.csv", parse_dates = ['started_at', 'ended_at'])

In [4]:
bikeshare_data.dtypes #Take a look at the data types

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id             float64
end_station_name              object
end_station_id               float64
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

#### Create database with the following code (Converted to markdown because only use once):

conn = psycopg2.connect("user=postgres password='password'")  
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);  
cursor = conn.cursor();  
sqlCreateDatabase = "create database bikesharedb;"  
cursor.execute(sqlCreateDatabase);  
cursor.close();

In [3]:
'''
Create a sql_alchemy engine to import pandas dataframes into SQL database. I am using this instead of psycopg2 
in order to save time versus using psycopg2 and CREATE TABLE or INSERT syntax. I imported it directly using SQL 
'''
dbname = 'bikesharedb' 
username='postgres'
password='password'
host='localhost'
db_conn = f'postgresql+psycopg2://{username}:{password}@{host}/{dbname}'

#### Create Table (Converted to markdown because only use once.)
table_name = 'bikeshare3'  
    
table = f'''CREATE TABLE {table_name} (  
ride_id INT PRIMARY KEY NOT NULL UNIQUE,  
rideable_type varchar(10),    
started_at timestamp,  
ended_at timestamp,  
start_station_name varchar(50),  
start_station_id  INT FOREIGN KEY NOT NULL,  
end_station_name  varchar(50),  
end_station_id INT FOREIGN KEY NOT NULL,  
start_lat REAL,  
start_lng REAL,  
end_lat   REAL ,  
end_lng   REAL,  
member_casual VARCHAR(15));  
'''    
conn = psycopg2.connect("user=postgres password='password'")  
db_cursor = conn.cursor()  
db_cursor.execute(table)  
conn.commit()  
db_cursor.close()      


db_cursor = conn.cursor()  
db_cursor.execute("COPY bikeshare3 FROM '202203-capitalbikeshare-tripdata.csv' WITH (FORMAT csv);")  
conn.commit()  
db_cursor.close()        


In [4]:
engine = create_engine(f'{db_conn}', echo=False)

#### Other way to send data to sql (Converted to Markdown because only use once)
bikeshare_data.to_sql('bikeshare3', con=engine, if_exists='replace', index=False) 

In [12]:
df = pd.read_sql_query("SELECT * FROM bikeshare3 LIMIT 5", engine) #can now read in Python
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,lat_change,long_change,duration,b
0,63DD0ABA5C512840,electric_bike,2022-03-29 18:55:57,2022-03-29 18:56:11,None,NaN,None,None,38.900000,-77.040000,38.90,-77.04,member,0.0,0.0,0 days 00:00:14,None
1,5258E72F8C5C66F3,electric_bike,2022-03-22 22:57:37,2022-03-22 23:16:33,None,NaN,None,None,38.800000,-77.040000,38.80,-77.04,casual,0.0,0.0,0 days 00:18:56,None
2,54DBDFE5135510AB,electric_bike,2022-03-22 22:31:00,2022-03-22 22:33:39,None,NaN,None,None,38.890000,-77.010000,38.89,-77.01,casual,0.0,0.0,0 days 00:02:39,None
3,90918421F47CA6C1,electric_bike,2022-03-22 22:26:13,2022-03-22 22:26:32,None,NaN,None,None,38.890000,-77.010000,38.89,-77.01,casual,0.0,0.0,0 days 00:00:19,None
4,E49D00F1DAD8BE20,docked_bike,2022-03-25 14:05:43,2022-03-25 16:07:16,4th St & Madison Dr NW,31288.0,None,None,38.890496,-77.017247,NaN,NaN,casual,NaN,NaN,0 days 02:01:33,None


In [17]:
df[df['ride_id'] == '5258E72F8C5C66F3']

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,lat_change,long_change,duration,b
1,5258E72F8C5C66F3,electric_bike,2022-03-22 22:57:37,2022-03-22 23:16:33,None,NaN,None,None,38.8,-77.04,38.8,-77.04,casual,0.0,0.0,0 days 00:18:56,None


#### How to add columns to a table (Converted to markdown because only use once
q = '''   
ALTER TABLE bikeshare3 ADD COLUMN duration REAL;   
ALTER TABLE bikeshare3 ADD COLUMN lat_change REAL;   
ALTER TABLE bikeshare3 ADD COLUMN long_change REAL;   
'''  
conn = psycopg2.connect(f'dbname=bikesharedb user=postgres password=password')  
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);  
db_cursor = conn.cursor()  
db_cursor.execute(q)    
conn.commit()  
db_cursor.close()  

In [8]:
df.dtypes #data types of columns. The columns I created above are now listed

ride_id                        object
rideable_type                  object
started_at             datetime64[ns]
ended_at               datetime64[ns]
start_station_name             object
start_station_id              float64
end_station_name               object
end_station_id                 object
start_lat                     float64
start_lng                     float64
end_lat                       float64
end_lng                       float64
member_casual                  object
lat_change                    float64
long_change                   float64
duration              timedelta64[ns]
b                              object
dtype: object

In [5]:
# it works!
df = pd.read_sql_query("SELECT * FROM bikeshare3 LIMIT 5", engine) #can now read in Python

In [6]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,lat_change,long_change,duration,b
0,1,testName,NaT,NaT,None,NaN,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaT,None
1,1F4FCB442E18DFAB,electric_bike,2022-03-19 14:20:15,2022-03-19 14:50:10,None,NaN,None,None,38.880000,-77.030000,38.88,-76.96,member,0.0,0.07,0 days 00:29:55,None
2,AD4FAF264C9EB1F2,docked_bike,2022-03-21 21:08:36,2022-03-30 10:00:12,21st St & Constitution Ave NW,31261.0,None,None,38.892459,-77.046567,NaN,NaN,casual,NaN,NaN,8 days 12:51:36,None
3,70DD2688B58DE3AD,electric_bike,2022-03-20 20:53:02,2022-03-20 20:58:38,None,NaN,None,None,38.900000,-77.010000,38.90,-76.99,casual,0.0,0.02,0 days 00:05:36,None
4,E5DB491266DC9F50,electric_bike,2022-03-07 17:40:54,2022-03-07 17:46:33,None,NaN,None,None,38.880000,-77.020000,38.88,-77.00,member,0.0,0.02,0 days 00:05:39,None


#### Error handling
I ran into an error because I designated the wrong type for duration above. It should be interval not real. Here I drop the column and then add it again as interval.

q = '''ALTER TABLE bikeshare3   
DROP COLUMN duration;'''  
conn = psycopg2.connect(f'dbname=bikesharedb user=postgres password=password')  
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);  
db_cursor = conn.cursor()  
db_cursor.execute(q)  
conn.commit()  
db_cursor.close()  

q = '''ALTER TABLE bikeshare3 ADD COLUMN duration INTERVAL;'''  
conn = psycopg2.connect(f'dbname=bikesharedb user=postgres password=password')  
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);  
db_cursor = conn.cursor()  
db_cursor.execute(q)  
conn.commit()  
db_cursor.close()  

q = '''   
UPDATE bikeshare3  
SET duration = CAST(ended_at AS TIMESTAMP) - CAST(started_at AS TIMESTAMP);  
UPDATE bikeshare3  
SET lat_change = end_lat - start_lat;  
UPDATE bikeshare3  
SET long_change = end_lng - start_lng;  
'''  

conn = psycopg2.connect(f'dbname=bikesharedb user=postgres password=password')  
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);  
db_cursor = conn.cursor()  
db_cursor.execute(q)  
conn.commit()  
db_cursor.close()  

In [8]:
'''
SELECT started_at, ended_at
FROM bikesharedb
LIMIT 5
'''
df = pd.read_sql_query("SELECT * FROM bikeshare3 LIMIT 5", engine) #can now read in Python

In [9]:
df #success - Setting duration to interval allowed me to engineer a new variable containing the duration!

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,lat_change,long_change,duration,b
0,5C6B106A8F184F26,electric_bike,2022-03-14 16:57:58,2022-03-14 17:06:41,1st & O St NW,31519.0,None,None,38.908682,-77.012388,38.92,-77.03,casual,0.011318,-0.017612,0 days 00:08:43,None
1,785A5820FB0919FB,electric_bike,2022-03-21 14:17:44,2022-03-21 14:23:56,None,NaN,None,None,38.920000,-77.050000,38.91,-77.06,casual,-0.010000,-0.010000,0 days 00:06:12,None
2,9FBE184BFBD7403F,classic_bike,2022-03-06 11:03:30,2022-03-07 12:03:24,6th & H St NE,31615.0,None,None,38.899972,-76.998347,NaN,NaN,casual,NaN,NaN,1 days 00:59:54,None
3,3AB2C3806C1AA363,electric_bike,2022-03-21 18:35:21,2022-03-21 18:36:15,None,NaN,None,None,38.900000,-77.000000,38.90,-77.00,casual,0.000000,0.000000,0 days 00:00:54,None
4,97C48A6A3E7A7901,electric_bike,2022-03-21 18:12:44,2022-03-21 18:14:44,None,NaN,None,None,38.920000,-77.030000,38.92,-77.03,casual,0.000000,0.000000,0 days 00:02:00,None


# Need to clean some data
It looks like there are some inaccurate data points. The end timestamp is before the start timestamp. We can drop these operations in the analysis but the data should be corrected at some time. 

In [13]:
q = '''
SELECT ride_id, duration, started_at, ended_at, AVG(duration) OVER(PARTITION BY start_station_id) as avg_duration_by_start_station
FROM bikeshare3
ORDER BY duration
LIMIT 10
'''
df = pd.read_sql_query(q, engine) #can now read in Python
df

,ride_id,duration,started_at,ended_at,avg_duration_by_start_station
0,F22BBAD63223D8D5,-1 days +23:57:06,2022-03-10 17:55:01,2022-03-10 17:52:07,0 days 00:23:32.226757
1,9FA4056C4B75F39C,-1 days +23:57:29,2022-03-31 14:04:13,2022-03-31 14:01:42,0 days 00:46:53.148067
2,4C1A26212690BA44,-1 days +23:58:16,2022-03-16 15:23:40,2022-03-16 15:21:56,0 days 00:14:29.193152
3,46F938EC00EA932C,-1 days +23:59:08,2022-03-28 16:51:45,2022-03-28 16:50:53,0 days 00:16:10.480916
4,B1EBF9DBA20A26B4,-1 days +23:59:25,2022-03-25 16:30:50,2022-03-25 16:30:15,0 days 00:30:26.920319
5,F8A94B9BEEA7BD07,-1 days +23:59:33,2022-03-26 15:45:50,2022-03-26 15:45:23,0 days 00:34:31.729335
6,016FE89395F179C3,-1 days +23:59:56,2022-03-05 14:11:50,2022-03-05 14:11:46,0 days 00:14:29.193152
7,3A0ECA9C06CEFF93,0 days 00:00:00,2022-03-14 07:53:50,2022-03-14 07:53:50,0 days 00:18:53.563185
8,9B3A2B7C8B16483B,0 days 00:00:00,2022-03-06 04:47:26,2022-03-06 04:47:26,0 days 00:13:08.038674
9,72D5ABB0C6F17B00,0 days 00:00:00,2022-03-21 14:52:57,2022-03-21 14:52:57,0 days 00:39:16.318092


We have three types of bikes that consumers can rent: classic_bike, docked_bike, and electric bike. The classic bike is the most popular rental followed by the electric bike and the docked_bike is the least popular rental. This could be due to inventory availability. 

In [14]:
q =''' 
SELECT DISTINCT rideable_type, COUNT(rideable_type) as num_rentals
FROM bikeshare3
GROUP BY rideable_type
'''
df = pd.read_sql_query(q, engine) 
df

,rideable_type,num_rentals
0,classic_bike,192354
1,docked_bike,19198
2,electric_bike,42698


The number of rentals per each station using a group by on start_station_id and an aggregate function in the select statement.

In [15]:
'''
SELECT start_station_id, COUNT(start_station_id) as num_rentals
FROM bikesharedb
GROUP BY start_station_id
ORDER BY num_rentals DESC
LIMIT 10 '''
df = pd.read_sql_query(q, engine)
df

,rideable_type,num_rentals
0,classic_bike,192354
1,docked_bike,19198
2,electric_bike,42698


#### NUMBER OF RENTALS PER DAY

In [16]:
q = '''
SELECT
  DATE(started_at) as StartDate,
  COUNT(ride_id) as CountOfRows 
FROM bikeshare3
GROUP BY 1
ORDER BY 1;
'''
df = pd.read_sql_query(q, engine)
df

,startdate,countofrows
0,2022-03-01,6928
1,2022-03-02,7888
2,2022-03-03,7055
3,2022-03-04,6717
4,2022-03-05,9692
5,2022-03-06,9340
6,2022-03-07,6650
7,2022-03-08,7598
8,2022-03-09,3891
9,2022-03-10,7577


#### We can also write the above query like so:

In [22]:
q ='''SELECT
  CAST(started_at AS DATE) as StartDate,
  COUNT(started_at) as CountOfRows 
FROM bikeshare3
GROUP BY CAST(started_at AS DATE)
ORDER BY CAST(started_at AS DATE);'''
df =  pd.read_sql_query(q, engine); #Cast to date removes timestamp

In [23]:
df #count of bike rentals per day

,startdate,countofrows
0,2022-03-01,6928
1,2022-03-02,7888
2,2022-03-03,7055
3,2022-03-04,6717
4,2022-03-05,9692
5,2022-03-06,9340
6,2022-03-07,6650
7,2022-03-08,7598
8,2022-03-09,3891
9,2022-03-10,7577


##### ONLY PULL WHERE DURATION IS GREATER THAN OR EQUAL TO AN INTERVAL OF 0 DAYS 0 HOURS AND 0 MINUTES.

In [17]:
df = pd.read_sql_query('''SELECT * 
                            FROM bikeshare3 
                            WHERE duration >= CAST('0 days 00:00:00' AS INTERVAL) 
                            ORDER BY duration''', engine)
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,lat_change,long_change,duration,b
0,6EE723BC73517BC3,classic_bike,2022-03-22 18:43:28,2022-03-22 18:43:28,New Jersey Ave & F St NW,31655.0,New Jersey Ave & F St NW,31655.0,38.897108,-77.011616,38.897108,-77.011616,member,0.000000,0.000000,0 days 00:00:00,None
1,8FD159FA30278F49,classic_bike,2022-03-25 15:04:16,2022-03-25 15:04:16,John McCormack Dr & Michigan Ave NE,31502.0,John McCormack Dr & Michigan Ave NE,31502.0,38.934600,-76.995500,38.934600,-76.995500,casual,0.000000,0.000000,0 days 00:00:00,None
2,29F300C29059C060,classic_bike,2022-03-21 15:41:50,2022-03-21 15:41:50,3rd & D St SE,31605.0,3rd & D St SE,31605.0,38.885100,-77.002300,38.885100,-77.002300,member,0.000000,0.000000,0 days 00:00:00,None
3,3C6BB8B171D0118D,classic_bike,2022-03-31 07:35:58,2022-03-31 07:35:58,1st & K St SE,31628.0,1st & K St SE,31628.0,38.878854,-77.005727,38.878854,-77.005727,casual,0.000000,0.000000,0 days 00:00:00,None
4,9B49A8A88FEF86E0,classic_bike,2022-03-16 18:56:06,2022-03-16 18:56:06,4th St & Madison Dr NW,31288.0,4th St & Madison Dr NW,31288.0,38.890496,-77.017246,38.890496,-77.017246,casual,0.000000,0.000000,0 days 00:00:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254238,AD4FAF264C9EB1F2,docked_bike,2022-03-21 21:08:36,2022-03-30 10:00:12,21st St & Constitution Ave NW,31261.0,None,NaN,38.892459,-77.046567,NaN,NaN,casual,NaN,NaN,8 days 12:51:36,None
254239,4AED037983B838FD,docked_bike,2022-03-13 19:34:25,2022-03-22 11:49:09,7th & R St NW / Shaw Library,31245.0,Good Hope Rd & 14th St SE,31803.0,38.912719,-77.022155,38.866611,-76.985238,casual,-0.046108,0.036917,8 days 16:14:44,None
254240,5AE580CF557383BD,docked_bike,2022-03-05 16:50:11,2022-03-17 00:05:08,8th & O St NW,31281.0,14th & Otis Pl NW,31131.0,38.908640,-77.022770,38.934405,-77.032687,casual,0.025765,-0.009917,11 days 07:14:57,None
254241,113BAF7ED46B30D8,docked_bike,2022-03-02 01:02:52,2022-03-17 11:39:56,Shady Grove Metro West,32045.0,King Farm Blvd & Piccard Dr,32038.0,39.119765,-77.166093,39.110314,-77.182669,casual,-0.009451,-0.016576,15 days 10:37:04,None


#### PostgreSQL functions

CREATE OR REPLACE FUNCTION total_rides()  
DELIMITER // #COMMENT: markdown reads $$ as italics  
RETURNS INTEGER AS /total/
DECLARE total integer;  
BEGIN  
SELECT COUNT(ride_id) INTO total FROM public.bikeshare3;  
RETURN total;  
END;  
/total/  
LANGUAGE plpgsql;  

In [19]:
pd.read_sql_query('''SELECT total_rides();''', engine)

,total
0,254251


##### Create a function to return the max duration of the rides in the dataset

CREATE OR REPLACE FUNCTION max_duration()  
DELIMITER //  #Comment: markdown reads $$ as italics  
RETURNS INTERVAL AS //maxd//  
DECLARE maxd INTERVAL;  
BEGIN  
SELECT MAX(duration) INTO maxd FROM public.bikeshare3;  
RETURN maxd;  
END;  
//maxd//
LANGUAGE plpgsql;  

In [ ]:
pd.read_sql_query('''SELECT max_duration();''', engine)